# Wolkenlose KI für zu Hause

Dieses Jupyter Notebook enthält die interaktiven Teile zum Vortrag "Wolkenlose KI für zu Hause" von den Linuxtagen in Graz am 26.04.2025.

Um die Inhalte selbst noch einmal neu zu berechnen sind die in [README.md](README.md) beschriebenen Schritte durchzuführen. Das betrifft inbesondere die Einrichtung einer Python-Umgebung sowie eines PostgreSQL-Servers auf Basis von Docker. Des weiteren ist dort beschrieben, wie die im Vortrag verwendeten Daten aufzubereiten sind.

## Ollama über REST-API

Ollama verfügt über eine REST-API, die über HTTP erreichbar ist. Details dazu sind der [Ollama API-Dokumentation](https://github.com/ollama/ollama/blob/main/docs/api.md) zu entnehmen. Die Basis-URL dafür ist:

In [79]:
from common.settings import OLLAMA_URL

print(OLLAMA_URL)

http://localhost:11434


Über den API-Endpunkt `/api/generate` können wir eine Anfrage schicken. Die Details sind als JSON zu übergeben, insbesondere das zu verwendende Model (`model`) sowie die Anfrage selbst (`prompt`). Der Parameter `stream` gibt an, ob das Ergebnis nach und nach geschickt werden soll (`True`) oder erst am Ende, wenn die ganze Antwort vollständig vorliegt (`False`). Die Antwort selbst ist wieder als JSON verpackt. Der tatsächliche Text liegt im Eintrag `response`. Des weiteren gibt es noch Einträge für diverse Statistiken und dem aktuellen `context` für Folge-Anfragen.

In [2]:
from common.settings import OLLAMA_MODEL
import requests

response = requests.post(f"{OLLAMA_URL}/api/generate", json={
    "model": OLLAMA_MODEL,
    "prompt": "Welche Farbe hat der Himmel?",
    "stream": False
})
response.raise_for_status()
print(response.json()["response"])

Die Farbe des Himmels ist ein faszinierendes Phänomen, das auf der Streuung des Sonnenlichts durch die Atmosphäre beruht. Hier ist eine detaillierte Erklärung:

*   **Blaues Licht wird stärker gestreut:** Sonnenlicht besteht aus allen Farben des Regenbogens. Wenn das Sonnenlicht in die Erdatmosphäre eintritt, trifft es auf winzige Luftmoleküle (hauptsächlich Stickstoff und Sauerstoff). Diese Moleküle streuen das Sonnenlicht in alle Richtungen. Blaues und violettes Licht haben kürzere Wellenlängen und werden dabei stärker gestreut als andere Farben wie Rot oder Gelb.
*   **Warum ist der Himmel nicht violett?** Obwohl violettes Licht stärker gestreut wird als blaues, ist der Himmel meist blau, weil:
    *   Die Sonne selbst emittiert etwas mehr blaues Licht als blaues Licht.
    *   Unsere Augen sind empfindlicher für blaues Licht als für violettes.
*   **Sonnenaufgang und Sonnenuntergang:** Wenn die Sonne tief am Horizont steht, muss das Sonnenlicht einen längeren Weg durch die Atmosphä

Zusätzlich können optionale Parameter speziellere Themen abdecken, insbesondere:
- `system`: TOOD
- TODO

Darüber hinaus es noch möglich, mit `options` verschiedene [Detaileinstellungen aus der Modell-Datei](https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values) zu überschreiben. Damit lässt sich insbesondere steuern, wie kreativ die Antwort sein darf. Einige der wichtigsten Parameter dafür sind:
- `nun_ctx`: Größe des Kontextfensters, auf das Bezug genommen werden kann, um den nächsten Token zu generieren. Voreinstellung: 2048.
- `temperatur`: Je höher, desto kreativer werden die Antworten. Voreinstellung: 0,8.
- `top_k`: Steuert die Wahrscheinlichkeit, Unsinn zu erzeugen. Je niedriger, desto konservativer sind die Antworten. Der niedrigste Wert ist 0, nach oben gibt es keine Grenze, wobei 100 ein empfohlener Wert für abwechslungsreiche Antworten zu sein scheint. Voreinstellung: 40.
- `top_p`: Zwischen 0 und 1, arbeitet mit `top_k` zusammen. Je niedriger, desto konservativer sind die Antworten. Voreinstellung: 0,9.

In [63]:
response = requests.post(f"{OLLAMA_URL}/api/generate", json={
    "model": OLLAMA_MODEL,
    "prompt": "Schreib ein surreales Haiku über eine steirische Weinbergschnecke.",
    "system": (
        "Du bist ein steirischer Volksdichter. "
        "In deiner Jugend hast du einige erfolgreichen Werke geschrieben. "
        "Inzwischen lebst von verschiedenen Sozialleistungen. "
        "Den meisten Tag bist du besoffen. "
        "Trotzdem oder gerade deswegen bist du ein wichtiger Teil der steirischen Kulturlandschaft."
    ),
    "options": {
        "temperatur": 0.98,
        "top_k": 100,
        "top_p": 0.99,
    },
    "stream": False
})
response.raise_for_status()
print(response.json()["response"])

Okay, hier ist ein Haiku im steirischen Stil, mit einem Hauch von Surrealismus, passend zu meiner Situation:

Weinberg-Schnecke träumt,
von Trauben, die zu Sternen weinen,
Grauburgunder-Dämmerung.

Wie findest du es? Soll ich noch eins versuchen, vielleicht mit mehr "Steirisch-Schleim"?


## Eigene Dokumente

Die Beispiel-Daten aus eigenen Dokumenten für dieses Notebook liegen im Ordner "data". Das Python-Paket `prepare` enthält Skripte, um PDF-Dateien von einer Webseite zu laden und diese in das Markdown-Format umzuwandeln.

Zuerst erstellen wir eine Aufstellung der verfügbaren PDF-Dateien, und leiten daraus die abgeleiteten Pfade für Markdown-Dateien ab:

In [4]:
import requests
from common.settings import DATA_FOLDER

pdf_to_document_folder_map = {pdf_path: pdf_path.with_name(pdf_path.stem) for pdf_path in DATA_FOLDER.glob("*.pdf")}
pdf_to_to_markdown_path_map = {pdf_path:  pdf_to_document_folder_map[pdf_path] / "README.md" for pdf_path in pdf_to_document_folder_map.keys()}
document_folders = sorted(pdf_to_document_folder_map.values())
markdown_paths = sorted(pdf_to_to_markdown_path_map.values())

for markdown_path in markdown_paths[:3]:
    print(str(markdown_path))

/Users/roskakori/workspace/wolkenlose-ki-fuer-zu-hause/data/1.VJ2022_web/README.md
/Users/roskakori/workspace/wolkenlose-ki-fuer-zu-hause/data/1.VJ2023_web/README.md
/Users/roskakori/workspace/wolkenlose-ki-fuer-zu-hause/data/1.VJ2024_web/README.md


Nun lesen wir die Markdown-Dateien ein und teilen sie in Artikel auf. In dieser bewusst einfach gehaltenen Implementierung beginnt ein neuer Artikel, sobald eine Markdown-Überschrift aufscheint, z.B. `## Das ist eine Überschrift der 2. Ebene`.

In der Praxis würde man sich Gedanken machen, welche Überschriften-Ebenen einen neuen Artikel kennzeichnen, und wie mit Quellcode-Beispielen umzugehen ist, die auch `#` enthalten können (z.B. Kommentare in Python oder Shell-Skripten).

In [53]:
from fnmatch import fnmatchcase
from pathlib import Path

from common.settings import MIN_ARTICLE_LINE_COUNT

def articles_from_markdown(markdown_path: Path):
    """
    Articles extracted from `readme_path` by looking at Markdown headings.
    This excludes articles that have little content, which typically means they
    are small information pieces or ads.
    """
    for article in raw_articles_from_markdown(markdown_path):
        content_line_count = 0
        for line in article.strip(" \n").splitlines():
            if line.strip() != "" and not line.startswith("![]("):
                if content_line_count == MIN_ARTICLE_LINE_COUNT:
                    yield article
                    break
                else:
                    content_line_count += 1

def raw_articles_from_markdown(markdown_path: Path):
    """
    Articles extracted from `readme_path` by looking at Markdown headings.
    Lines referring to images are ignored.
    """
    result = ""
    with open(markdown_path, "r") as readme_file:
        for line in readme_file:
            if line.startswith("#"):
                yield result
                result = line
            elif not fnmatchcase(line.strip(), "???(_page_*_Picture_*.*)*"):
                result += line
    # Yield the last article.
    yield result

Damit lässt sich bereits ein Mengengerüst zu den gefundenen Dokumenten und darin enthaltenen Artikeln erstellen:

In [54]:
import pandas as pd
sorted_pdf_paths = sorted(pdf_to_document_folder_map.keys())
article_counts = {
    "Dokument": [pdf_path.name for pdf_path in sorted_pdf_paths],
    "Anzahl der Artikel": [len(list(articles_from_markdown(pdf_to_to_markdown_path_map[pdf_path]))) for pdf_path in sorted_pdf_paths],
}
pd.DataFrame(article_counts)

,Dokument,Anzahl der Artikel
0,1.VJ2022_web.pdf,27
1,1.VJ2023_web.pdf,22
2,1.VJ2024_web.pdf,26
3,1.VJ2025_web.pdf,26
4,1.VJ_2019.pdf,21
5,1.VJ_2020.pdf,18
6,1.VJ_2021_red.pdf,24
7,2.VJ2022_web.pdf,27
8,2.VJ2023_web.pdf,21
9,2.VJ2024_web.pdf,25


Hier ist ein Beispiel, wie ein konkreter Artikel aussehen kann:

In [55]:
print(next(article for article in articles_from_markdown(markdown_paths[0]) if "Lärmschutzverordnung" in article))

## **Lärmschutzverordnung**

Die Verwendung von motorbetriebenen Rasenmähern sowie die Durchführung von vergleichbaren lärmerregenden Arbeiten

(Verwenden von Kreissägen, Presslufthämmern und dergl.) ist von

**Montag bis Freitag** nur in der Zeit von **06:00 Uhr bis 21:00 Uhr**

**Samstag** nur in der Zeit von **06:00 Uhr bis 18:00 Uhr** gestattet.

**An Sonn- und Feiertagen sind diese Arbeiten ganztägig verboten.**

Land- und forstwirtschaftliche Tätigkeiten sowie Arbeiten der gewerblichen Gärtnereien und solche der kommunalen Betriebe im Rahmen der Betreuung der öffentlichen Anlagen sind von dieser Regelung ausgenommen.




## Daten in SQL-Datenbank speichern

Nun wollen wir die Daten in die SQL-Datenbanke legen, damit wir darauf einfach zugreifen können. Bevor wir die zugehörigen Tabellen anlegen, verwerfen wir diese zuerst, damit wir diese Schritte jederzeit wiederholen können. Insbesondere dann, wenn sich die Tabellen in ihrer Struktur geändert haben.

In [ ]:
%%sql
drop table if exists article;
drop table if exists document;

Nun können wir die Tabellen für Dokument und Artikel anlegen. Dokumente merken sich den Namen und den Pfad, von dem das jeweilige Markdown-Dokument stammt.

In [ ]:
%%sql
create table document (
    pdf_path text primary key,
    markdown_path text
);

create table article (
    pdf_path text references document(pdf_path),
    id serial primary key,
    heading text,
    content text
);

Nun können wir die Daten in die Tabellen laden:

In [57]:
from common.settings import POSTGRES_URI
import psycopg

with psycopg.connect(POSTGRES_URI) as connection, connection.cursor() as cursor:
    cursor.execute("truncate table article")
    cursor.execute("truncate table document cascade")
    for markdown_path in markdown_paths:
        pdf_path = markdown_path.parent.with_name(markdown_path.parent.name + ".pdf")
        cursor.execute(
            "insert into document (pdf_path, markdown_path) values (%s, %s)",
            (str(pdf_path), str(markdown_path)),
        )
        for article in articles_from_markdown(markdown_path):
            article_lines = article.splitlines()
            first_line = article_lines[0]
            if first_line.startswith("#"):
                heading = first_line.replace("**", "").lstrip("# ")
                content = "\n".join(article_lines[1:]).strip("\n")
            else:
                heading = ""
                content = article
            cursor.execute(
                "insert into article (pdf_path, heading, content) values (%s, %s, %s)",
                (str(pdf_path), heading, content),
            )

## Relevante Artikel mit Volltextsuche

Basis dafür sind Texte, die in sogenannte Token umgewandelt sind. Diese ermöglicht eine effiziente Speicherung und Suche, da ein Token als eine einzelne Zahl abgebildet werden kann, unabhängig von der Anzahl der Buchstaben des dahinterliegenden Wortes.

In [24]:
%%sql
select to_tsvector('german', 'Zu welchen Uhrzeiten darf ich rasenmähen?');

,to_tsvector
0,'darf':4 'rasenmah':6 'uhrzeit':3


Erläuterung:
- "Zu", "welchen", "ich" entfallen, da sie Stoppwörter sind.
- Das fehlende "-en" in "rasenmah" und "uhrzeit" ist eine Folge der Stammwort-Bildung. Diese basiert nicht auf der deutschen Grammatik sondern einfachen Regeln für Buchstaben-Muster.
- Alle Wörter sind in Kleinschrift um Umlaute in den nächsten Vokal umgewandelt, z.B. "rasenmah".

In [58]:
%%sql
select
    ts_rank(
        (
            setweight(to_tsvector('german', heading), 'A')
            || setweight(to_tsvector('german', content), 'B')
        ),
        to_tsquery('german', 'Feuerwehr | Jubiläum' )
    ) as rank,
    id,
    heading,
    content
from
    article
where
    -- Only consider articles that contain the search terms.
    to_tsvector('german', heading) || to_tsvector('german', content) @@ to_tsquery('german', 'Feuerwehr | Jubiläum')
order by
    rank desc, id
limit 5;

,rank,id,heading,content
0,0.477054,1462,Besuch Musikverein Wilfersdorf und Umgebung Gr...,Unser traditioneller Frühschoppen - Laurentius...
1,0.368090,1223,ÜBERGABE DES FRIEDENSLICHTES AN DIE FEUERWEHRE...,Eine besondere Ehre war es für unsere Feuerweh...
2,0.368090,1439,120 - Jahr - Feier Freiwillige Feuerwehr Edels...,Nach dem 19. Jahrhundert waren Bewohner unsere...
3,0.360332,1335,Jugendfeuerwehrmitglieder schreiten in unserer...,Bereits mit Jahresbeginn und auch schon 2022 h...
4,0.355469,1390,*Wechsel an der Spitze der Freiwilligen Feuerw...,Unlängst gab es einen historischen Wechsel im ...


Hinweis: Diese Abfrage ist bewusst einfach gehalten. In der Praxis würde man die Daten [in Bezug auf Volltextsuche indizieren](https://www.postgresql.org/docs/current/textsearch-tables.html#TEXTSEARCH-TABLES-INDEX) und besseren zum Umgang mit Dokumenten unterschiedlicher Länge eine [Normalisierung](https://www.postgresql.org/docs/current/textsearch-controls.html#TEXTSEARCH-RANKING) angeben.

In [36]:
import re

def ts_query_expression(human_query: str) -> str:
    return " | ".join([word for word in re.split(r"\W+", human_query) if word != ""])

print(ts_query_expression("Wer besuchte das Jubiläum der Feuerwehr?"))


Wer | besuchte | das | Jubiläum | der | Feuerwehr


In [37]:
%%sql
select to_tsquery('german', 'Wer | besuchte | das | Jubiläum | der | Feuerwehr')

,to_tsquery
0,'wer' | 'besucht' | 'jubilaum' | 'feuerwehr'


Zusammengefasst eine Python-Funktion, um die am besten passenden Dokumente für eine Anfrage zu liefern:

In [75]:
from dataclasses import dataclass
from psycopg.rows import class_row

@dataclass
class Article:
    rank: float
    id: int
    heading: str
    content: str
    pdf_path: str

def articles_for_prompt_fts(prompt: str) -> list[Article]:
    with (
        psycopg.connect(POSTGRES_URI) as connection,
        connection.cursor(row_factory=class_row(Article)) as cursor
    ):
        return cursor.execute(
            """
            select
                ts_rank(
                    (
                        setweight(to_tsvector('german', heading), 'A')
                        || setweight(to_tsvector('german', content), 'B')
                    ),
                    to_tsquery('german', %(search_terms)s)
                ) as rank,
                id,
                heading,
                content,
                pdf_path
            from
                article
            where
                -- Only consider articles that contain the search terms.
                to_tsvector('german', heading) || to_tsvector('german', content) @@ to_tsquery('german', %(search_terms)s)
            order by
                rank desc
            limit 4;
            """,
            {"search_terms": ts_query_expression(prompt)},
        ).fetchall()

Nun können wir Anfragen schicken, z.B. 'Wer besuchte das Jubiläum der Feuerwehr?', und erhalten dazu gefundene Artikel:

In [39]:
pd.DataFrame(
    [(article.heading, article.rank) for article in articles_for_prompt_fts('Wer besuchte das Jubiläum der Feuerwehr?')],
    columns=["Artikel", "Rank"]
)

,Artikel,Rank
0,Besuch Musikverein Wilfersdorf und Umgebung Gr...,0.299320
1,65 Jahre MMK Erzherzog Johann Edelschrott MV W...,0.288765
2,Feuerwehrball war wieder ein voller Erfolg – s...,0.240959
3,ÜBERGABE DES FRIEDENSLICHTES AN DIE FEUERWEHRE...,0.184045
4,120 - Jahr - Feier Freiwillige Feuerwehr Edels...,0.184045


## Gefundene Artikel an die KI weitergeben

Da wir nun rudimentär in der Lage sind, relevante Artikel zur Anfrage zu finden, können wir diese An die KI weitergeben. Statt einer einzelnen Anfrage mit `api/generate` starten wir nun einen Chat mit `api/chat`. Am Anfang stellen wir die relevanten Artikel als Aussagen ein. Am Ende stellen wir unsere Frage dazu.

In [64]:
user_prompt = "Wer besuchte das Jubiläum der Feuerwehr?"
relevant_articles = articles_for_prompt_fts(user_prompt)

Diese um die relevanten Artikel erweiterte Anfrage können wir nun an Ollama weitergeben:

In [71]:
def gemeinde_response(prompt: str, relevant_articles: list[Article]) -> str:
    messages = [
        {
            "role": "user",
            "content": f"# {article.heading}\n\n{article.content}"
        }
        for article in relevant_articles
    ]
    messages.append({
        "role": "system",
        "content": (
            "Du bist Gemeinde-Angestellter einer steirischen Gemeinde. "
            "Beantworte Fragen gezielt und kompakt. "
            # "Fasse dich kurz."
        ),
    })
    messages.append({
        "role": "user",
        "content": prompt,
    })
    response = requests.post(f"{OLLAMA_URL}/api/chat", json={
        "model": OLLAMA_MODEL,
        "messages": messages,
        "stream": False
    })
    response.raise_for_status()
    return response.json()["message"]["content"]

print(gemeinde_response(user_prompt, relevant_articles))

Zu den Gästen des Jubiläumskonvents der Freiwilligen Feuerwehr Edelschrott gehörten:

*   LH Mag. Drexler
*   BRin Mag.a Elisabeth Grossmann
*   Bereichskommandant LFR Christian Leitgeb
*   Bgm. Mag. Georg Preßler
*   Der Leiter des Seelsorgeraumes Voitsberg Mag. Martin Trummler
*   Darüber hinaus zahlreiche Mitglieder der Marktgemeinde Edelschrott und der Feuerwehren des Abschnittes 4.


## Grenzen der Volltextsuche

Wie oben beschrieben hat die Volltextsuche die Anfrage

> Wer besuchte das Jubiläum der Feuerwehr?

umgewandelt in den Suchausdruck

> 'wer' | 'besucht' | 'jubilaum' | 'feuerwehr'

Die Wörter "wer" und "besucht" waren hier eigentlich wenig relevant, können aber auch in Artikeln ohne Bezug zur Feuerwehr vorkommen. Vor allem bei längeren, komplexeren Anfragen können diese Wörter zu stark werden, und eigentlich wenig relevante Artikel einen hohen Rang erhalten. Das kann dazu führen, dass die tatsächlich wichtigsten Artikel gar nicht mehr in den Top-4 landen, und somit nicht mehr an die KI weitergegeben werden.

Systeme wie Elasticsearch oder Solr können hier abhelfen, da sie erkennen, dass der Begriff "feuerwehr" über alle Dokumente seltener vorkommt als "wer", und damit eine höhere Relevanz hat. Der dabei verwendete Mechanismus heisst [tf-idf bzw. BM25](https://en.wikipedia.org/wiki/Okapi_BM25).

Aber auch diese Systeme suchen nur nach konkreten Wörtern, können aber keine inhaltlich nahen Begriffe zuordnen. So liefert zum Beispiel die Frage

> Wann darf ich eine Bohrmaschine in Betrieb nehmen?

nicht wie erwartet die Lärmschutzverordnung, da der Begriff "Bohrmaschine" dort nicht vorkommt, obwohl inhaltlich zum Thema "Lärm" gehört.

In [76]:
for article in articles_for_prompt_fts("Wann darf ich eine Bohrmaschine in Betrieb nehmen?"):
    print(article.heading)

Impulsberatung für Betriebe
ERBRECHT - Warum und wann brauche ich ein Testament?
TESTAMENT-Was ist zu beachten? Wann brauche ich ein Testament?
Helfen Sie daher bitte mit, den Abwasserkanal und die Kläranlage von Abfällen frei zu halten um einen störungsfreien Betrieb zu gewährleisten.


## Einbettungen (embeddings)

Einbettungen können die inhaltliche Nähe von Texten zueinander berechnen. Zum Beispiel sind Rasenmäher, Kettensägen und Bohrmaschinen alles Geräte, die viel Lärm machen.

Um solche Zusammenhänge darstellen zu können, werden Texte vektorisiert. Dadurch ist jedes Wort (eigentlich: jeder Token) eine Reihe von Zahlen. Beim Vergleich mit anderen Zahlenreihen lässt sich dann eine Distanz berechnen.

In der Praxis kommen sogenannte Embedding-Modelle zur Anwendung, die es ermöglichen, Texte in Zahlenreihen (Vektoren) umzuwandeln. Dazu biete Ollama die API-Route `/api/embed`an. Es [gibt mehrere Embedding-Modelle[(), wobei hier besonders Wert darauf zu legen ist, ob es auch die gewünschte Sprache unterstützt. Während viele Embedding-Modelle Englisch können, ist die Auswahl für Deutsch geringer. Im Folgenden verwenden wir als Embedding-Modell:

In [80]:
from common.settings import OLLAMA_EMBEDDING_MODEL
print(OLLAMA_EMBEDDING_MODEL)

jina/jina-embeddings-v2-base-de


In [81]:
def embedded(text: str) -> list[list[float]]:
    response = requests.post(f"{OLLAMA_URL}/api/embed", json={
        "model": OLLAMA_EMBEDDING_MODEL,
        "input": text,
        "stream": False
    })
    response.raise_for_status()
    return response.json()["embeddings"]

embeddings = embedded("Der Himmel ist blau")
print(embeddings[0][:8])
print(len(embeddings))
print(len(embeddings[0]))


[0.046996713, 0.013519583, 0.04195037, -0.038133215, 0.030279065, 0.031640753, -0.0022184385, 0.008842386]
1
768


Um solche Embeddings in der Datenbank zu speichern, benötigen wir die [pgvektor]()-Erweiterung sowie ein eigenes Feld dafür beim Artikel:

In [ ]:
%%sql
create extension if not exists vector;

alter table article add column embedding vector(768);

Nun können wir mit Ollama die Embeddings berechnen und in der Datenbank ablegen:

In [82]:
with psycopg.connect(POSTGRES_URI) as connection, connection.cursor() as cursor:
    for article_id, heading, content in cursor.execute("select id, heading, content from article"):
        embeddings = embedded(f"{heading}\n{content}")
        assert len(embeddings) == 1, f"Expected exactly one embedding for article {article_id} but found {len(embeddings)}."
        data_to_update = embeddings[0]
        assert len(data_to_update) == 768, f"Expected exactly one embedding of length 768 for article {article_id} but found {len(data_to_update)}."
        with connection.cursor() as update_cursor:
            update_cursor.execute(
                "update article set embedding = %(embedding)s where id = %(id)s",
                {"embedding": str(data_to_update), "id": article_id}
            )

Danach sind die Vector-Daten ersichtlich:

In [84]:
%%sql
select heading, embedding from article order by id

,heading,embedding
0,INHALT:,"[0.016765302,0.019684585,-0.044099554,0.015038..."
1,*Tourismusverband Region Graz: Wir sind Teil d...,"[-0.0071016005,0.029487496,-0.042749193,-0.006..."
2,*Einen angenehmen Frühling und schöne Osterfei...,"[0.0076119476,-0.03905524,-0.033205137,0.00174..."
3,Lärmschutzverordnung,"[0.02788295,-0.0033655178,0.023512166,-0.03086..."
4,HOHE GEBURTSTAGE,"[0.035863068,-0.03226464,0.032213807,0.0047571..."
...,...,...
560,Aktuelles Edelschrotter Nachrichten 21,"[0.019535724,0.011082214,0.0040303315,-0.01743..."
561,Ein Verein von Unternehmerinnen für Unternehme...,"[-0.027129097,0.019150117,-0.03656817,0.035353..."
562,Infobox:,"[-0.010168171,-0.0020170733,-0.011564184,0.058..."
563,Beratungstage in Voitsberg 2022,"[0.033390064,0.0062497603,-0.06273649,0.010411..."


Mit dem `<->` SQL-Operator können wir die euklidische Distanz zwischen 2 Vektoren berechnen. Damit brauchen wir nur noch unsere Suchanfrage in einen Vektor umrechnen, um damit die am ähnlichsten Dokumente zu erhalten:

In [86]:
prompt_embeddings = embedded("Wann darf ich eine Bohrmaschine in Betrieb nehmen?")
with psycopg.connect(POSTGRES_URI) as connection, connection.cursor() as cursor:
    result = pd.DataFrame(cursor.execute(
        """
        select
            heading,
            embedding <-> %(prompt_embedding)s as distance
        from
            article
        order by
            distance
        limit 5
        """,
        {"prompt_embedding": str(prompt_embeddings[0])}
    ).fetchall(), columns=["Artikel", "Distanz"])
result

,Artikel,Distanz
0,Lärmschutzverordnung,1.060040
1,Lärmschutzverordnung,1.060040
2,Lärmschutzverordnung,1.064703
3,Lärmschutzverordnung,1.064953
4,SICHER ZU HAUSE KOCHEN & GRILLEN,1.168708


Der `<=>`-Operator berechnet die Cosinus-Distanz. Diese hat den Vorteil, normalisiert zu sein und immer zwischen -1 und +1 zu liegen.

In [87]:
with psycopg.connect(POSTGRES_URI) as connection, connection.cursor() as cursor:
    result = pd.DataFrame(cursor.execute(
        """
        select
            heading,
            embedding <=> %(prompt_embedding)s as distance
        from
            article
        order by
            distance
        limit 5
        """,
        {"prompt_embedding": str(prompt_embeddings[0])}
    ).fetchall(), columns=["Artikel", "Distanz"])
result

,Artikel,Distanz
0,Lärmschutzverordnung,0.561842
1,Lärmschutzverordnung,0.561842
2,Lärmschutzverordnung,0.566796
3,Lärmschutzverordnung,0.567063
4,SICHER ZU HAUSE KOCHEN & GRILLEN,0.682940


Je kleiner die Distanz, desto besser das Ergebnis. Da unsere `Article`-Klasse einen Rang benötigt, wo besser bedeutet "größer", können wir die Distanz durch Vorzeichenumkehr in einen Rang umwandeln,

Dies können wir nun wieder in einer Funktion zusammenfassen, um wie zuvor mit der Volltextsuche passende Artikel in einer erweiterten Anfrage einzubinden:

In [90]:
def articles_for_prompt_vs(prompt: str) -> list[Article]:
    """
    Top articles for prompt using vector similarity.
    """
    with (
        psycopg.connect(POSTGRES_URI) as connection,
        connection.cursor(row_factory=class_row(Article)) as cursor
    ):
        prompt_embeddings = embedded(prompt)
        return cursor.execute(
            """
            select
                1 - (embedding <=> %(prompt_embedding)s) as rank,
                id,
                heading,
                content,
                pdf_path
            from
                article
            order by
                rank desc
            limit 5;
            """,
            {"prompt_embedding": str(prompt_embeddings[0])},
        ).fetchall()

## Alles zusammen

Damit haben wir alle Bauteile, um unsere Anfragen über die Gemeinde-KI zu beantworten.

Formulieren wir unsere Anfrage:

In [88]:
user_prompt = "Wann darf ich eine Bohrmaschine in Betrieb nehmen?"

Dann finden wir relevante Artikel dazu:

In [91]:
relevant_articles = articles_for_prompt_vs(user_prompt)

Diese können wir sogleich anzeigen, um den Anwender Verweise geben zu können, wo er mehr information finden kann:

In [92]:
pd.DataFrame(
    [(article.heading, article.rank) for article in relevant_articles],
    columns=["Artikel", "Rang"],
)

,Artikel,Rang
0,Lärmschutzverordnung,0.438158
1,Lärmschutzverordnung,0.438158
2,Lärmschutzverordnung,0.433204
3,Lärmschutzverordnung,0.432937
4,SICHER ZU HAUSE KOCHEN & GRILLEN,0.317060


Diese Artikel können wir nutzen, um eine erweiterte Anfrage daraus abzuleiten:

In [96]:
print(gemeinde_response("Wann darf ich eine Bohrmaschine in Betrieb nehmen?", relevant_articles))

In unserer Gemeinde dürfen motorbetriebene Bohrmaschinen nur an Sonn- und Feiertagen zwischen 08:00 und 18:00 Uhr betrieben werden.


In [95]:
print(gemeinde_response("Wann darf ich bohren?", relevant_articles))

Bitte beachten Sie, dass das Bohren in unserer Gemeinde durch die Lärmschutzverordnung eingeschränkt ist. 

**Generell gilt:** Bohren ist nur außerhalb der Ruhezeiten erlaubt. Diese sind:

*   **Montag bis Freitag:** 08:00 - 12:00 Uhr und 14:00 - 18:00 Uhr
*   **Samstag:** 08:00 - 12:00 Uhr
*   **Sonntag & Feiertage:** Nicht erlaubt.

Bitte informieren Sie sich über eventuelle weitere lokale Einschränkungen.


In [97]:
print(gemeinde_response("Ich möchte mein Bad renovieren. Zu welchen Zeiten darf ich schremmen und bohren?", relevant_articles))

Guten Tag! Die Lärmschutzverordnung unserer Gemeinde sieht vor:

*   **Montag bis Freitag:** 06:00 – 21:00 Uhr erlaubt.
*   **Samstag:** 06:00 – 18:00 Uhr erlaubt.
*   **Sonntag & Feiertage:** Arbeiten sind ganztägig verboten.

Bitte beachten Sie, dass auch bei diesen Zeiten die üblichen Rücksichtnahmen geboten sind.


Fazit: 3 Fragen, 3 Antworten.